In [ ]:
################################## hr 과 lr 을 train / valid 로 분리
from glob import glob
import os
import shutil
import random

base_paths = ['/home/jovyan/playGround/ziippy/dl_data/open_sr/train/hr/', 
             '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/lr/']
new_train_paths = ['/home/jovyan/playGround/ziippy/dl_data/open_sr/train/hr_split_train/',
                  '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/lr_split_train/']
new_valid_paths = ['/home/jovyan/playGround/ziippy/dl_data/open_sr/train/hr_split_valid/',
                  '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/lr_split_valid/']

random.seed(2022)

for index, base_path in enumerate(base_paths):
    new_train_path = new_train_paths[index]
    new_valid_path = new_valid_paths[index]
    
    print(new_train_path)
    print(new_valid_path)
    
    shutil.rmtree(new_train_path, ignore_errors=True)
    os.makedirs(new_train_path, exist_ok=True)
    shutil.rmtree(new_valid_path, ignore_errors=True)
    os.makedirs(new_valid_path, exist_ok=True)

    file_list = glob(base_path + '*.png')
    file_count = len(file_list)
    print(file_count)
    
    file_list.sort()

    #random.shuffle(file_list)

    train_ratio = 0.975
    train_index = int(file_count * train_ratio)
    print(train_index)
    print(file_count - train_index)

    for file in file_list[:train_index]:
        filename = file.split('/')[-1]
        shutil.copy(file, new_train_path+filename)
        #print(filename)    
    print('split for train done')

    for file in file_list[train_index:]:
        filename = file.split('/')[-1]
        shutil.copy(file, new_valid_path+filename)
        #print(filename)
    print('split for valid done')
    
print('all done')

In [11]:
import os

hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/hr_split_train/'
lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/lr_split_train/'

new_hr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/new_hr/'
new_lr_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/new_lr/'
new_hr_valid_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/new_hr_valid/'
new_lr_valid_path = '/home/jovyan/playGround/ziippy/dl_data/open_sr/train/new_lr_valid/'
path_splitter = '/'

os.makedirs(new_hr_path, exist_ok=True)
os.makedirs(new_lr_path, exist_ok=True)
os.makedirs(new_hr_valid_path, exist_ok=True)
os.makedirs(new_lr_valid_path, exist_ok=True)

target_width = 512
target_height = 512

In [14]:
target_width = 512
target_height = 512

from PIL import Image
from glob import glob
import os, shutil
from tqdm import tqdm

import numpy as np
import cv2

# remove old files
filelist = glob(os.path.join(new_hr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_lr_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_hr_valid_path, "*"))
for f in filelist:
    os.remove(f)
filelist = glob(os.path.join(new_lr_valid_path, "*"))
for f in filelist:
    os.remove(f)

# variables
total_count = 0
designated_valid_count = 60
current_count = 0
do_each = False
train_ratio = 1.0
using_original = True
   
# hr
filelist = glob(os.path.join(hr_path, "*.png"))
filelist.sort()

total_count = len(filelist)
train_index = int(total_count * train_ratio)

if using_original is True:
    org_hr_list = glob(os.path.join(hr_path, "*.png"))
    for file in tqdm(org_hr_list[:train_index]):
        filename = file.split('/')[-1]
        shutil.copy(file, new_hr_path + filename)
    print('copy original hr done')

    org_lr_list = glob(os.path.join(lr_path, "*.png"))
    for file in tqdm(org_lr_list[:train_index]):
        filename = file.split('/')[-1]
        shutil.copy(file, new_lr_path + filename)
    print('copy original lr done')

for _, file in enumerate(tqdm(filelist[:train_index])):
    filename = file.split(path_splitter)[-1]
    filename_without_ext = filename.split('.')[0]   

    ### resize or filter
    modes = ['nearest', 'box', 'bilinear', 'hamming', 'bicubic', 'lanczos', 'blur', 'gaussian_blur', 'median_blur', 'bilateral_filter']
    applys = [True, False, True, False, True, True, False, True, False, True]
    
    #
    do_modes = []
    for index, mode in enumerate(modes):
        if applys[index] is True:
            do_modes.append(mode)
    if do_each is False: # 파일마다 한번씩만 round robin
        do_mode = do_modes[current_count % len(do_modes)]
        do_modes.clear()
        do_modes.append(do_mode)
        
    # train or valid
    for_valid = False
    if current_count + designated_valid_count >= train_index:
        for_valid = True

    target_hr_path = new_hr_path
    target_lr_path = new_lr_path
    if for_valid is True:
        target_hr_path = new_hr_valid_path
        target_lr_path = new_lr_valid_path
    
    for mode in do_modes:
        if mode in ['nearest', 'box', 'bilinear', 'hamming', 'bicubic', 'lanczos']:
            # resize
            img = Image.open(file)
            if mode == 'nearest':
                img_resize = img.resize((target_width, target_height), Image.NEAREST)
            elif mode == 'box':
                img_resize = img.resize((target_width, target_height), Image.BOX)
            elif mode == 'bilinear':
                img_resize = img.resize((target_width, target_height), Image.BILINEAR)
            elif mode == 'hamming':
                img_resize = img.resize((target_width, target_height), Image.HAMMING)
            elif mode == 'bicubic':
                img_resize = img.resize((target_width, target_height), Image.BICUBIC)
            elif mode == 'lanczos':
                img_resize = img.resize((target_width, target_height), Image.LANCZOS)
                
            # copy hr
            new_file_hr = target_hr_path + filename_without_ext + "_" + mode + ".png"
            shutil.copy(file, new_file_hr)
            # copy lr
            new_file_lr = target_lr_path + filename_without_ext + "_" + mode + ".png"
            img_resize.save(new_file_lr)
        
            #print(filename + ' ' + mode + ' done')
        else:
            img = cv2.imread(file)
            resize_img = cv2.resize(img, (target_width, target_height))   
            if mode == 'blur':
                dst = cv2.blur(resize_img, (5,5))
            elif mode == 'gaussian_blur':
                dst = cv2.GaussianBlur(resize_img, (5,5), 0)
            elif mode == 'median_blur':
                dst = cv2.medianBlur(resize_img, 5)
            elif mode == 'bilateral_filter':
                dst = cv2.bilateralFilter(resize_img, 9, 75, 75)
            else:
                continue
            
            # copy hr
            new_file_hr = target_hr_path + filename_without_ext + "_" + mode + ".png"
            shutil.copy(file, new_file_hr)
            # copy lr
            new_file_lr = target_lr_path + filename_without_ext + "_" + mode + ".png"
            cv2.imwrite(new_file_lr, dst)

            #print(filename + ' ' + mode + ' done')
    
    current_count += 1
    #if current_count > 10:
    #    break
    
print('all done')

### 컨볼루션
#img = cv2.imread(hr_path + filename)
#resize_img = cv2.resize(img, (target_width, target_height))
#kernel = np.ones((5,5),np.float32)/25
#dst = cv2.filter2D(resize_img, -1, kernel)
#new_file = lr_path + '7_convolution_' + filename
#cv2.imwrite(new_file, dst)

100%|██████████| 1599/1599 [00:12<00:00, 123.11it/s]


copy original hr done


100%|██████████| 1599/1599 [00:06<00:00, 233.57it/s]


copy original lr done


100%|██████████| 1599/1599 [04:38<00:00,  5.73it/s]

all done
